### This notebook searches all possible cointegration basket for a given list of currency pairs.

In [1]:
import os, sys
import warnings
warnings.filterwarnings("ignore")

# Add SDevPy location to path
sdev_path = r"C:\\Code\\SDev.Python"
if sdev_path not in sys.path:
    if os.path.exists(sdev_path):
        sys.path.append(sdev_path)
    else:
        print("Path to SDevPy not found: " + sdev_path)

from sdevpy.cointegration import coint_trading as ct
from sdevpy.cointegration import data_io as myio
from sdevpy.cointegration import model_settings as settings

root = r"C:\\temp\\sdevpy\\cointegration"

### User specifications
Load a dataset including (CHF or not)

In [ ]:
# Dataset without CHF
ticker_list = ['EURUSD Curncy', 'GBPUSD Curncy', 'AUDUSD Curncy', 'NZDUSD Curncy', 
               'JPYUSD Curncy', 'CADUSD Curncy', 'SGDUSD Curncy', 'CNHUSD Curncy']

df_fx_spot = myio.read_fx_spot_data(os.path.join(root, 'data/bloomberg fx data sheet.xlsx'))

print(df_fx_spot.head())

start_list = ['2010-08-23', '2010-10-23',
              '2011-01-23', '2011-04-23', '2011-07-23', '2011-10-23',
              '2012-01-23', '2012-04-23', '2012-07-23', '2012-10-23',
              '2013-01-23', '2013-04-23', '2013-07-23', '2013-10-23',
              '2014-01-23', '2014-04-23', '2014-07-23', '2014-10-23',
              '2015-01-23', '2015-04-23', '2015-07-23', '2015-10-23',
              '2016-01-23', '2016-04-23', '2016-07-23', '2016-10-23',
              '2017-01-23', '2017-04-23', '2017-07-23', '2017-10-23',
              '2018-01-23', '2018-04-23', '2018-07-23', '2018-10-23',
              '2019-01-23', '2019-04-23', '2019-07-23', '2019-10-23',
             ]
             
TODAY = '2024-01-19'
output_xls = os.path.join(root, 'cointegration_search_on_' + TODAY + '.xlsx') 

FX spot data set is from = 2010-08-24 to 2024-10-04
            EURUSD Curncy  GBPUSD Curncy  AUDUSD Curncy  NZDUSD Curncy  \
Dates                                                                    
2010-08-23         1.2657         1.5512         0.8913         0.7068   
2010-08-24         1.2627         1.5396         0.8817         0.7023   
2010-08-25         1.2659         1.5458         0.8840         0.6995   
2010-08-26         1.2716         1.5528         0.8863         0.7029   
2010-08-27         1.2763         1.5529         0.8989         0.7109   

            JPYUSD Curncy  CADUSD Curncy  CHFUSD Curncy  NOKUSD Curncy  \
Dates                                                                    
2010-08-23       0.011743       0.950390       0.960799       0.160210   
2010-08-24       0.011919       0.942063       0.967961       0.158466   
2010-08-25       0.011823       0.943752       0.970309       0.158448   
2010-08-26       0.011841       0.945269       0.976562    

In [ ]:
# Dataset with CHF
ticker_list = ['EURUSD Curncy', 'GBPUSD Curncy', 'AUDUSD Curncy', 'NZDUSD Curncy', 
               'JPYUSD Curncy', 'CADUSD Curncy', 'SGDUSD Curncy', 'CNHUSD Curncy', 'CHFUSD Curncy']

df_fx_spot = myio.read_fx_spot_data(os.path.join(root, 'data/bloomberg fx data sheet.xlsx'))

start_list = ['2015-07-23', '2015-10-23',
              '2016-01-23', '2016-04-23', '2016-07-23', '2016-10-23',
              '2017-01-23', '2017-04-23', '2017-07-23', '2017-10-23',
              '2018-01-23', '2018-04-23', '2018-07-23', '2018-10-23',
              '2019-01-23', '2019-04-23', '2019-07-23', '2019-10-23',
             ]
             
TODAY = '2024-01-19'
output_xls = os.path.join(root, 'cointegration_search_on_' + TODAY + '.xlsx') 

### Performing the Johansen Test on all basket combinations

In [6]:
res_df = ct.johansen_compute_all_baskets(start_list, TODAY, ticker_list, df_fx_spot)
df_coint = ct.filter_cointegration_basket_using_trace_10(res_df)
print('Number of baskets = ' + str(len(res_df))  )
print('Number of cointegrated baskets = ' + str(len(df_coint))  )

df_coint_diag = ct.compute_johansen_test_diag_for_all_coint_baskets(df_coint, df_fx_spot)

100%|██████████| 38/38 [00:27<00:00,  1.38it/s]


Number of baskets = 7980
Number of cointegrated baskets = 5035


100%|██████████| 5035/5035 [00:49<00:00, 101.87it/s]


### Filter for cointegrated basket with certain criteria. Then compute a few extra diagnostics for the selected basket and export to Excel

In [ ]:
SHOW_BASKET_WITH_SD_ABOVE = 1.5
SHOW_BASKET_WITH_SHARPE_ABOVE = 1.0
SHOW_BASKET_WITH_HALF_LIFE_IN_DAYS_BELOW = 100

df_filtered = ct.filter_cointegration_basket_using_SD_threshold(df_coint_diag, SHOW_BASKET_WITH_SD_ABOVE)  
df_filtered = ct.filter_cointegration_basket_using_1m_1SD_in_USD(df_filtered)
df_filtered = ct.filter_cointegration_basket_using_Sharpe_Ratio(df_filtered, SHOW_BASKET_WITH_SHARPE_ABOVE)
df_filtered = ct.filter_cointegration_basket_using_half_life_in_days(df_filtered, SHOW_BASKET_WITH_HALF_LIFE_IN_DAYS_BELOW)

df_filtered = ct.compute_johansen_stability_diagnostics(df_filtered, df_fx_spot, settings.DataFreq.DAILY)
df_filtered = ct.compute_historical_min_max_SD_diagnostics(df_filtered, df_fx_spot)

In [ ]:
df_output_formatted = myio.format_cointegration_seach_output(df_filtered)
df_output_formatted.to_excel(output_xls)

print(df_filtered[['SD Current']])
print(df_filtered[['what you should trade']])
print(df_filtered[['currency pairs']])
print(df_filtered[['half life in days']]) 
print(df_filtered[['half life Sharpe Ratio']])
print(df_filtered[['RSI 14']])
print(df_filtered[['Stop Loss in SD']])
print(df_filtered[['From']])